In [ ]:
import pandas as pd
d = pd.read_excel("../EUR/Energy/Polak-Kwartier-20190101.xlsx", names=["time", "kWh"], converters= {"Datum-tijd tot": pd.to_datetime}, skiprows=4, index_col=0)
# d

In [ ]:
# What effect do vars have on co2with 
import os
def read_energy_files(dir):                                                                                                  
    r = []
    for root, dirs, files in os.walk(dir):
        if dirs and not dirs[0].isdigit():
            continue    
        for year_dir in dirs:
            building_usages = os.path.join(root, os.path.join(year_dir,"EAN 26728"))
            for year_root, year_dirs, year_files in os.walk(building_usages):
                for building_dir in year_dirs:
                    for building_root, building_dirs, building_files in os.walk(os.path.join(year_root, building_dir)):
                        print("dir:",building_dir, "files", building_files)
                    
                        for name in building_files[-1:]: 
                            d = pd.read_excel(os.path.join(building_root, name), names=["time", "kWh"], converters= {"Datum-tijd tot": pd.to_datetime}, skiprows=4, index_col=0)
                            d["building_name"] = building_dir
                            r.append(d)
    return r 
energy_usages = pd.concat(read_energy_files(r"EUR\Energy"))
# energy_usages.to_csv("EUR/Integrated/energy_usages.csv")

In [ ]:
import re

solar_col_dic = {
    "Elektriciteit Consumptie (kWh)": "elec_consumed(kWh)",
    "Elektriciteit Productie (kWh)": "elec_poduced(kWh)",
    "Blindverbruik Consumptie (kVARh)": "blinduse_consumed(kVARh)",
    "Blindverbruik Productie (kVARh)" : "blinduse_produced(kVARh)"
}

solar_building_name_dic = {
    "vd Goot" : "Van der Goot",
    "Poilak" : "Polak",
    "polak" : "Polak",
}

def read_solar_energy_files(dir):                                                                                                  
    r = []
    for root, dirs, files in os.walk(dir):
        for file in files:
            skip_rows = 4
            if "Geb" in file or "Mandeville" in file: 
                skip_rows = 3
            # if "Geb" not in file: 
            #     continue
            # d = pd.read_excel(os.path.join(root, file), names=["time", "elec_consumed", "elec_poduced", "blinduse_consumed(kVARh)", "blinduse_produced(kVARh)"], converters= {"Datum-tijd tot": pd.to_datetime}, skiprows=4, index_col=0)

            d = pd.read_excel(os.path.join(root, file), converters= {"Datum-tijd tot": pd.to_datetime}, skiprows=skip_rows, index_col=0)
            print(d.columns)
            d = d.rename(columns=solar_col_dic)
            d.index.name = "time"

            building_name = re.findall(r'(.*)\s+Export-Kwartier-.*.xlsx', file)[0]
            building_name = building_name.strip()
            print(building_name)
            print(d.columns)
            if building_name in solar_building_name_dic:
                building_name = solar_building_name_dic[building_name]
            d["building_name"] = building_name
            r.append(d)
    return r 
solar_energy = pd.concat(read_solar_energy_files(r"..\EUR\Kwartierwaarden PV opbrengst"))
solar_energy.building_name.unique()
solar_energy.columns


In [ ]:
solar_energy.to_csv("../EUR/Integrated/solar_energy.csv")

In [ ]:
d["month"] = pd.DatetimeIndex(d.index).month 
d["year"] = pd.DatetimeIndex(d.index).year
d = d[d["year"]==2019]
d.tail(4)

monthly_means = d.groupby(pd.Grouper(level='time', freq='D'))["kWh"].sum()
monthly_means = monthly_means.groupby(pd.Grouper(level='time', freq='M')).sum()
# daily_mean_for_year = monthly_means.mean()
print(monthly_means)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

fig = plt.figure(figsize=(32, 12))
# fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(8,8))
# axs= axs.ravel()
# month_grps = d.groupby("month")

# for ax, (i, sub) in zip(axs, month_grps):
#     sub["kWh"].plot(kind="line", x="time", legend=True, title=i, ax=ax)
# monthly_means = d.resample("M").mean()with 
# print(monthly_means)
monthly_means = d.groupby(pd.Grouper(level='time', freq='D'))["kWh"].sum()
monthly_means = monthly_means.groupby(pd.Grouper(level='time', freq='M')).mean()
daily_mean_for_year = monthly_means.mean()
print(daily_mean_for_year)
# print(monthly_means)

for i in range(1, (12+1)):
    ax = fig.add_subplot(4, 3, i)
    # print(f"len is {len(d)}")
    plot_d = d[d["month"]==i]
    # monthly_mean = monthly_means[monthly_means["month"]==i]
    monthly_mean = monthly_means[monthly_means.index.month==i]
    mean = monthly_mean.iloc[0]
    # print(mean)
    # print(i, monthly_mean)
    # print(i, monthly_mean.head(1))
    # print(f"len is {len(plot_d)}")
    # print(plot_d.tail(2))
    # day_grps = plot_d.groupby(pd.Grouper(level='time', freq='D')).sum()
    day_grps = plot_d.groupby(pd.Grouper(level='time', freq='D'))["kWh"].sum()

    # print(i)
    # print(day_grps)
    day_grps.plot(kind="line", y="kWh", ax=ax)
    # monthly_mean.plot(kind="line", ax=ax)
    ax.axhline(y=mean, color="g", alpha=0.5, linestyle="-")
    ax.axhline(y=daily_mean_for_year, alpha=1, color="y", linestyle="dotted")
    ax.legend(["daily kWh", "local daily avg", "global daily avg"])
    ax.set_xlabel("Day", labelpad=-3)
    ax.set_ylabel("Electricity Usage (kWh)")
    ax.grid()
    # ax.lin
plt.tight_layout()